In [ ]:
import ipyvolume as ipv


import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np

import diskwarp
from diskwarp import helper

%config InlineBackend.figure_format = 'retina'

## General Definitions of disk surface and warp parameters

### Basic data

In [ ]:
# Calculate the interfaces and cell centers of the disk surface

r_i = np.linspace(0.1, 150, 201)
r_c = 0.5 * (r_i[1:] + r_i[:-1])

surf     = helper.get_surface(r_i)
p0_i     = surf['points_i']
p0_c     = surf['points_c']
ri       = surf['ri']
rc       = surf['rc']
phic     = surf['phic']
phii     = surf['phii']
nr, nphi = p0_c.shape[:-1]

# Calculate the velocities of all centers and edges

v0_c = (p0_c[:, :, 0]**2 + p0_c[:, :, 1]**2)**-0.25
v0_c = v0_c[None, :, :] * np.array([-np.sin(phic), np.cos(phic), np.zeros_like(phic)])
v0_c = np.moveaxis(v0_c, 0, 2)

v0_i = (p0_i[:, :, 0]**2 + p0_i[:, :, 1]**2)**-0.25
v0_i = v0_i[None, :, :] * np.array([-np.sin(phii), np.cos(phii), np.zeros_like(phii)])
v0_i = np.moveaxis(v0_i, 0, 2)

### Parameters

In [ ]:
i_warp = 60.0
phi_twist = 0.0

inc_out = 80.0
PA_out  = -45.0

### Function to warp the data

In [ ]:
def warp_data(i_warp, i_out, PA_in, PA_out):
    # Define the warp (inclination for each ring): logistic function from $i=0$ outside transitioning to `warp_max` at radius `r0` over a transition width of `dr`.

    warp_c = helper.warp(r_c, i_in=i_warp, i_out=i_out)
    warp_i = helper.warp(r_i, i_in=i_warp, i_out=i_out)

    # Define each radius' twist

    twist_i = helper.twist(r_i, PA_in=PA_in, PA_out=PA_out)
    twist_c = helper.twist(r_c, PA_in=PA_in, PA_out=PA_out)

    #p1_c = diskwarp.fmodule.apply_matrix2d(p0_c, warp_c, twist_c, np.deg2rad(inc_obs), np.deg2rad(PA_obs), np.deg2rad(azi))
    #v1_c = diskwarp.fmodule.apply_matrix2d(v0_c, warp_c, twist_c, np.deg2rad(inc_obs), np.deg2rad(PA_obs), np.deg2rad(azi))
    p1_i = diskwarp.fmodule.apply_matrix2d(p0_i, warp_i, twist_i)
    #v1_i = diskwarp.fmodule.apply_matrix2d(v0_i, warp_i, twist_i, np.deg2rad(inc_obs), np.deg2rad(PA_obs), np.deg2rad(azi))
    
    return p1_i.T

### Set colors

In [ ]:
C = Normalize()(ri[:, :].T)
col = plt.cm.RdBu(C)

### Visualize

In [ ]:
X, Y, Z = warp_data(i_warp, inc_out, phi_twist, PA_out)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='3d')
ax.view_init(90, -90)
surf = ax.plot_surface(X, Y, Z, facecolors=col, shade=False, linewidth=0)
ax.set_xlim(-r_i[-1], r_i[-1])
ax.set_ylim(-r_i[-1], r_i[-1])
ax.set_zlim(-r_i[-1], r_i[-1])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

In [ ]:
import numpy as np
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets


class Scene:
    def __init__(self, view=(0, 0, 0), hkl=(1,1,1)):
        self.view = view
        self.figure = ipv.figure()
        R = 150.
        ipv.xlim(-R, R)
        ipv.ylim(-R, R)
        ipv.zlim(-R, R)
        ipv.style.box_off()
        self.xyz = p0_i.T
        self.surface = ipv.plot_surface(*self.xyz, color=col)
        ipv.show()
        self.ui = widgets.interact(
            self.update,
            i_warp=(-90.,90.),
            phi_twist=(-90., 90.),
            inc_out=(0.0, 180.0),
            PA_out=(-90.0, 90),
        )
        self.ui.widget.children[0].continuous_update = False
        self.ui.widget.children[1].continuous_update = False

    def update(self,
               i_warp = 60.0,
               phi_twist = 0.0,
               inc_out = 80.0,
               PA_out  = -45.0,
              ):
        x, y, z = warp_data(i_warp, inc_out, phi_twist, PA_out)
        self.surface.x = x.flatten()
        self.surface.y = y.flatten()
        self.surface.z = z.flatten()

scene = Scene()